<a href="https://colab.research.google.com/github/kcj4800/Tensorflow_practice/blob/main/Titanic_Survived.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# 세로열이 많을땐 feature columns를 사용하여 자동화하자
import pandas as pd
import os

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/titanic/train.csv')
print(data)
print(data.isnull().sum())

# 평균값으로 빈칸 채우기
age_avg = data['Age'].mean()
print(age_avg)
data['Age'].fillna(value=30, inplace = True)

# 최빈값으로 빈칸 채우기
Embarked_mode = data['Embarked'].mode()
print(Embarked_mode)
data['Embarked'].fillna(value = 'S', inplace = True)
print(data.isnull().sum())
# Embarked

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [40]:
import tensorflow as tf


Y = data.pop('Survived')
X = data
# model.fit(ds_batch, validation_data = (valX, valY), suffle = True, epochs = 20) 처럼 하기 위해서는
# sklearn을 이용하는 방법이 있다. 
# 참고. 악플분석

# from tensorflow.keras.preprocessing.sequence import pad_sequences

# X = pad_sequences(train_seq, maxlen=100)

# from sklearn.model_selection import train_test_split

# trainX, valX, trainY, valY = train_test_split( X, Y, test_size = 0.2, random_state = 42)

# print(len(trainX))
# print(len(valX))
print(len(X))
from sklearn.model_selection import train_test_split
trainX, valX, trainY, valY = train_test_split( X, Y, test_size = 0.2, random_state = 42)
print(len(trainX))
print(len(valX))

ds = tf.data.Dataset.from_tensor_slices( (dict(X), Y))

# csv데이터를 집어넣을때 feature column을 이용하기 위해서는 이러한 형태의 dataset을 만들어야 한다.

# print(ds)

# for i, l in ds.take(1):
#   print(i, l)


891
712
179


In [41]:
# 첫 레이어는 tf.keras.layers.Dense() 대신 tf.keras.layers.DenseFeatures()로 한다.
# nomalizing : 숫자를 0 ~ 1 사이로 압축해준다.
feature_columns = []
numeric_columns = ['Fare', 'Parch', 'SibSp']

for i in numeric_columns:
  def normalizer_func(x):
    최소 = data[i].min()
    최대 = data[i].max()
    return (x - 최소) / (최대 - 최소) # x = 최댓값일 경우 = 1, x = 최솟값일 경우 = 0 => 0 ~ 1 사이로 압축된다.
  feature_columns.append(tf.feature_column.numeric_column(i, normalizer_fn = normalizer_func))

# feature_columns.append(tf.feature_column.numeric_column('Fare', normalizer_fn = 노말라이저함수))
# feature_columns.append(tf.feature_column.numeric_column('Parch'))
# feature_columns.append(tf.feature_column.numeric_column('SibSp'))

# feature_columns.append(tf.feature_column.numeric_column('Age'))

# bucketized_column
Age = tf.feature_column.numeric_column('Age')
Age_bucket = tf.feature_column.bucketized_column(Age, boundaries = [10, 20, 30, 40, 50, 60])

feature_columns.append(Age_bucket)


# 각 컬럼을 어떻게 전처리할지 먼저 생각해보기
								
# 숫자로 집어넣은것 : Fare, SibSp, Parch : numeric_column
# 뭉퉁그려서 집어넣을거 : Age : bucketized_column
# 종류 몇개 없는 카테고리화 해서 집어넣을거 : Sex, Embarked Pclass : indicator_column
# 종류가 넘 많은 카테고리 : Ticket : embedding_column - 원하는 행렬로 무작위 숫자를 설정 한뒤 학습 진행하면서 적절한 숫자를 자동으로 찾는다.
# 스킵해도 될 내용 : Name, PassengerId
# 정답 컬럼 : Survived

print(Age)
print(feature_columns)

NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
[NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function normalizer_func at 0x7fe449ba9120>), NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function normalizer_func at 0x7fe449ba9bd0>), NumericColumn(key='SibSp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function normalizer_func at 0x7fe449ba9e10>), BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(10, 20, 30, 40, 50, 60))]


In [42]:
# sequence_categorical_column_with_vocabualary_list() -> categorical_column_with_vocabulary_list()
# a = ['Sex', 'Embarked', 'Pclass']
# for i in a:
#   vocab = data[i].unique() # 'Sex' column 의 unique_words
#   print(vocab)
#   cat = tf.feature_column.categorical_column_with_vocabulary_list(i, vocab )
#   one_hot = tf.feature_column.indicator_column(cat) # one_hot 인코딩해준다.
#   feature_columns.append( one_hot )  

vocab = data['Sex'].unique() # 'Sex' column 의 unique_words
print(vocab)
cat = tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocab )
one_hot = tf.feature_column.indicator_column(cat) # one_hot 인코딩해준다.
feature_columns.append( one_hot )

vocab = data['Embarked'].unique() # 'Embarked' column 의 unique_words
print(vocab)
cat = tf.feature_column.categorical_column_with_vocabulary_list('Embarked', vocab )
one_hot = tf.feature_column.indicator_column(cat) # one_hot 인코딩해준다.
feature_columns.append( one_hot )

vocab = data['Pclass'].unique() # 'Pclass' column 의 unique_words
print(vocab)
cat = tf.feature_column.categorical_column_with_vocabulary_list('Pclass', vocab )
one_hot = tf.feature_column.indicator_column(cat) # one_hot 인코딩해준다.
feature_columns.append( one_hot )


# embedding
vocab = data['Ticket'].unique() # 'Ticket' column 의 unique_words
print(vocab)
cat = tf.feature_column.categorical_column_with_vocabulary_list('Ticket', vocab )
one_hot = tf.feature_column.embedding_column(cat, dimension = 9) # 몇 차원의 행렬로 임베딩 할것인지 정해준다.
feature_columns.append( one_hot )


['male' 'female']
['S' 'C' 'Q']
[3 1 2]
['A/5 21171' 'PC 17599' 'STON/O2. 3101282' '113803' '373450' '330877'
 '17463' '349909' '347742' '237736' 'PP 9549' '113783' 'A/5. 2151'
 '347082' '350406' '248706' '382652' '244373' '345763' '2649' '239865'
 '248698' '330923' '113788' '347077' '2631' '19950' '330959' '349216'
 'PC 17601' 'PC 17569' '335677' 'C.A. 24579' 'PC 17604' '113789' '2677'
 'A./5. 2152' '345764' '2651' '7546' '11668' '349253' 'SC/Paris 2123'
 '330958' 'S.C./A.4. 23567' '370371' '14311' '2662' '349237' '3101295'
 'A/4. 39886' 'PC 17572' '2926' '113509' '19947' 'C.A. 31026' '2697'
 'C.A. 34651' 'CA 2144' '2669' '113572' '36973' '347088' 'PC 17605' '2661'
 'C.A. 29395' 'S.P. 3464' '3101281' '315151' 'C.A. 33111' 'S.O.C. 14879'
 '2680' '1601' '348123' '349208' '374746' '248738' '364516' '345767'
 '345779' '330932' '113059' 'SO/C 14885' '3101278' 'W./C. 6608'
 'SOTON/OQ 392086' '343275' '343276' '347466' 'W.E.P. 5734' 'C.A. 2315'
 '364500' '374910' 'PC 17754' 'PC 17759' '23191

In [31]:
print(type(trainX), type(trainY), type(valX), type(valY))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


In [43]:
# DenseFeautres에 들어가는 데이터를 미리 출력 해보고 싶을 때,

ds_batch = ds.batch(32)

next(iter(ds_batch))[0]

feature_layer = tf.keras.layers.DenseFeatures(tf.feature_column.numeric_column('Fare'))
feature_layer(next(iter(ds_batch))[0])

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[  7.25  ],
       [ 71.2833],
       [  7.925 ],
       [ 53.1   ],
       [  8.05  ],
       [  8.4583],
       [ 51.8625],
       [ 21.075 ],
       [ 11.1333],
       [ 30.0708],
       [ 16.7   ],
       [ 26.55  ],
       [  8.05  ],
       [ 31.275 ],
       [  7.8542],
       [ 16.    ],
       [ 29.125 ],
       [ 13.    ],
       [ 18.    ],
       [  7.225 ],
       [ 26.    ],
       [ 13.    ],
       [  8.0292],
       [ 35.5   ],
       [ 21.075 ],
       [ 31.3875],
       [  7.225 ],
       [263.    ],
       [  7.8792],
       [  7.8958],
       [ 27.7208],
       [146.5208]], dtype=float32)>

In [44]:
import numpy as np

model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns), # 딕셔너리 데이터를 넣는 첫번째 레이어에는 위에서 만든 feature_columns 리스트를 넣어준다.
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2), # overfitting 완화기능 : 윗 레이어의 노드 중 20%를 제거해주세요.
    tf.keras.layers.Dense(1, activation = 'sigmoid'),
])


model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# model.fit(trainX, trainY, validation_data = (valX, valY),  epochs = 20)

# dataset을 batch를 뽑아서 넣아야한다. 그렇지 않으면 cannot have rank 0 에러가 난다 

ds_batch = ds.batch(32) # 데이터 셋 자료를 32개씩 자르고 싶을때 .batch(32)를 이용해 자를 수 있다.

model.fit(ds_batch, shuffle=True, epochs=20)

# model.fit(ds_batch, validation_split = 0.2, shuffle = True, epochs = 20) 처럼 할수 없다.
# batch dataset에는 validation_split을 할수가 없다. numpy_array나 tensor에만 validation_split=0.2를 할수있다.
# model.fit(ds_batch, validation_data = (valX, valY), suffle = True, epochs = 20) 처럼 하기 위해서는
# sklearn을 이용하는 방법이 있다. 
# 참고. 악플분석

# from tensorflow.keras.preprocessing.sequence import pad_sequences

# X = pad_sequences(train_seq, maxlen=100)

# from sklearn.model_selection import train_test_split

# trainX, valX, trainY, valY = train_test_split( X, Y, test_size = 0.2, random_state = 42)

# print(len(trainX))
# print(len(valX))

Epoch 1/20


28/28 [==============================] - 4s 34ms/step - loss: 0.6203 - accuracy: 0.7059
Epoch 2/20
28/28 [==============================] - 0s 12ms/step - loss: 0.5295 - accuracy: 0.7407
Epoch 3/20
28/28 [==============================] - 0s 13ms/step - loss: 0.4717 - accuracy: 0.7845
Epoch 4/20
28/28 [==============================] - 0s 12ms/step - loss: 0.4499 - accuracy: 0.7991
Epoch 5/20
28/28 [==============================] - 0s 8ms/step - loss: 0.4227 - accuracy: 0.8182
Epoch 6/20
28/28 [==============================] - 0s 7ms/step - loss: 0.4087 - accuracy: 0.8227
Epoch 7/20
28/28 [==============================] - 0s 7ms/step - loss: 0.3946 - accuracy: 0.8294
Epoch 8/20
28/28 [==============================] - 0s 7ms/step - loss: 0.3678 - accuracy: 0.8496
Epoch 9/20
28/28 [==============================] - 0s 7ms/step - loss: 0.3324 - accuracy: 0.8653
Epoch 10/20
28/28 [==============================] - 0s 7ms/step - loss: 0.3069 - accuracy: 0.8743
Epoch 11/20
28/28 [=======